# Quantum Development Kit Samples<br>Diagnostics: Visualizing Quantum Programs

## Preamble

We'll first open the [Microsoft.Quantum.Diagnostics namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics) so that its functions and operations are available throughout the notebook.

In [1]:
open Microsoft.Quantum.Diagnostics;

## What is a quantum program?

Quantum programs are really **classical programs** that can send instructions to and get measurement results back from simulators and quantum devices.

As an example of this, **Q#** is a classical language designed to make it easier to write, test, and run quantum programs:

- High-level
- Focused on algorithms
- Portable across simulators and hardware
- Quantum-focused features
- Diagnostics to help test and understand

Q# is part of the **Microsoft Quantum Development Kit (QDK)**: [aka.ms/get-started-qdk](https://aka.ms/get-started-qdk)

## Why is visualization important for quantum programs?

When running algorithms on quantum hardware, we **cannot observe the quantum state** prior to measurement.

Even when running in a simulator, it may be **hard to gain intuition about the state** purely from wavefunction amplitudes.

**Visualization helps with intuition** and may make it easier to:
- understand the behavior of a quantum algorithm
- identify bugs in its implementation

## Example: Teleportation

**Quantum teleportation** is a textbook example of a simple quantum algorithm.

It uses **entanglement and measurements** to "teleport" the exact quantum state of a source qubit to a target qubit.

Let's use this algorithm to demonstrate a number of visualization techniques provided by Q# and the QDK:

   - visualizing quantum **operations**
   - visualizing quantum **states**
   - visualizing an **execution path** for a quantum algorithm
   - visualizing the **step-by-step operation** of a quantum program

## Visualizing a Q# operation using `DumpOperation`

We begin by **defining a Q# operation** called `PrepareEntangledState`.

We can **visualize this operation as a unitary matrix** by using `DumpOperation`.

In [2]:
operation PrepareEntangledState(qubits : Qubit[]) : Unit is Adj {
    H(qubits[0]);
    CNOT(qubits[0], qubits[1]);
}

operation VisualizePrepareEntangledState() : Unit {
    DumpOperation(2, PrepareEntangledState);
}

PrepareEntangledState, VisualizePrepareEntangledState

In [3]:
%simulate VisualizePrepareEntangledState

Real:
[[0.7071067811865477, 0.7071067811865477, 0, 0], 
[0, 0, 0.7071067811865477, -0.7071067811865477], 
[0, 0, 0.7071067811865477, 0.7071067811865477], 
[0.7071067811865477, -0.7071067811865477, 0, 0]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0]]

()

## Visualizing quantum states within `Teleport` using `DumpRegister`


In [4]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Random;

operation Teleport() : Unit {
    using ((source, intermediate, target) = (Qubit(), Qubit(), Qubit())) {
    
        // Store a random message in the source qubit and visualize its state.
        let randomAngle = DrawRandomDouble(0.0, 2.0*PI());
        Rx(randomAngle, source);
        Message("State of source qubit:");
        DumpRegister((), [source]);
    
        // Create some entanglement that we can use to send our message.
        PrepareEntangledState([intermediate, target]);

        // Entangle the source qubit with the intermediate qubit.
        CNOT(source, intermediate);
        H(source);

        // Measure the qubits and decode the message by applying corrections on the target qubit.
        if (M(source) == One) { Z(target); }
        if (M(intermediate) == One) { X(target); }
        
        // Visualize the current state of the target qubit.
        Message("State of target qubit:");
        DumpRegister((), [target]);
        Reset(target);
    }
}

Teleport

If we now simulate the `Teleport` operation, we can **verify that the quantum state is teleported correctly** each time it is run.

In [5]:
%simulate Teleport

State of source qubit:


|0⟩	0.9198280183216668 + 0𝑖
|1⟩	0 + -0.39232182798110443𝑖

State of target qubit:


|0⟩	0.9198280183216668 + 0𝑖
|1⟩	0 + -0.39232182798110454𝑖

()

## Visualizing an execution path of a Q# operation with `%trace`

- Q# operations may consist of **complex programming paradigms** (e.g. recursion, loops)
- **Hard to represent** as a quantum circuit
- Visualize an **execution path** instead!

For a simple program, the execution path looks exactly like a circuit:

In [6]:
operation MeasureEntangledState() : Result[] {
    using (qubits = Qubit[2]) {
        H(qubits[0]);
        CNOT(qubits[0], qubits[1]);
        return [M(qubits[0]), M(qubits[1])];
    }
}

MeasureEntangledState

In [7]:
%trace MeasureEntangledState

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Complex programs are **modular** and will likely contain **nested operations**:

In [8]:
open Microsoft.Quantum.Arrays;

operation MeasureEntangledStateModular() : Result[] {
    using (qubits = Qubit[2]) {
        PrepareEntangledState(qubits);
        return ForEach(M, qubits);
    }
}

MeasureEntangledStateModular

With `%trace`, we can visualize the program at a high level and **zoom in** to each operation as desired:

In [9]:
%trace MeasureEntangledStateModular

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

## Execution paths are non-deterministic!

For example, an operation that uses a **"repeat-until-success" loop** may have very different execution paths depending on measurement results:

In [10]:
operation MeasureUntilOne() : Unit {
    mutable result = Zero;
    using (q = Qubit()) {
        repeat {
            H(q);
            set result = M(q);
        }
        until (result == One);
    }
}

MeasureUntilOne

In [11]:
%trace MeasureUntilOne

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

## Visualizing an execution path for `Teleport`


The execution path for `Teleport` will differ based on the results of the two measurements.

By repeating `%trace`, we see that the `X` and `Z` operations **only sometimes occur** on the target qubit.

In [12]:
%trace Teleport

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

## Stepping through a Q# operation with `%debug`

- Classical programming tools typically have **debuggers**, which run one line of code at a time and allow inspection of variable state at each step.
- For small Q# programs, the `%debug` command allows one to **observe how the quantum state changes** as a quantum program runs.

Here we step through an operation that **prepares and measures an entangled state**:

In [13]:
%debug MeasureEntangledState

Starting debug session for MeasureEntangledState...


Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Finished debug session for MeasureEntangledState.


One, One

## Stepping through `Teleport`

Here we step through the `Teleport` operation, which tracks the quantum state of all three qubits:

In [14]:
%debug Teleport

Starting debug session for Teleport...


Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Finished debug session for Teleport.


()

## Summary

We have demonstrated several visualization techniques provided by Q# and the QDK:

   - visualizing quantum **operations**
   - visualizing quantum **states**
   - visualizing an **execution path** for a quantum algorithm
   - visualizing the **step-by-step operation** of a quantum program

A **variety of visualization tools** can be useful for quantum programs. Not one-size-fits-all!

Visualization is an important way for students and researchers to **gain intuition** about quantum algorithms and **understand the operation** of quantum programs.

## Epilogue

In [15]:
%version

Component,Version
iqsharp,0.13.20102604
Jupyter Core,1.4.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
